# Classic spring-mass-damper system
The spring-mass-damper system is a classic second-order differential equation that describes the dynamics of a mass attached to a spring and damper. The system can be represented by the following differential equation:

\begin{equation}
m\ddot{x}(t) + c\dot{x}(t)+kx(t)=F(t)
\end{equation}
where:
* $m$ is the mass attached to the spring and damper.
* $c$ is the damping coefficient.
* $k$is the spring constant.
* $x(t)$ is the displacement of the mass from its equilibrium position as a function of time.
* $\dot{x}(t)$ is the velocity of the mass (first derivative of displacement with respect to time).
* $\ddot{x}(t)$ is the acceleration of the mass (second derivative of displacement with respect to time).
* $F(t)$ is the external force applied to the system.

## Breaking down the equation
1. **Mass Term**: $m\ddot{x}(t)$
    * This term represents the inertia of the mass. It resists changes in motion (acceleration).
2. **Damping Term**: $c\dot{x}(t)$
    * The damping term is proportional to the velocity and opposes the motion. It models energy dissipation in the system (like friction).
3. **Spring Term**: $kx(t)$
    * The spring term is proportional to the displacement and acts to restore the mass to its equilibrium position.
4. **External Force**: $F(t)$
    * This is any external force acting on the system, which could be a function of time.

## Defining initial conditions
To solve this system, we need initial conditions such as:
\begin{equation}
x(0) = x_0 \\
\dot{x}(0)=v_0
\end{equation}

where $x_0$ is the initial displacement, and $v_0$ is the initial velocity.

# DeepXDE
It is a framework for solving differential equations using neural networks.

## Define the differential equation

In [14]:
def ode_system(t, y):
    """
    ODE system for the spring-mass-damper.
    - y1 = x(t): Displacement
    - y2 = v(t): Velocity
    """
    x, v = y[:, 0:1], y[:, 1:2]  # Splitting the output y into x and v
    dx_dt = v  # Velocity is the derivative of displacement
    dv_dt = (-c * v - k * x + F(t)) / m  # Derived from Newton's second law
    return [dx_dt, dv_dt]

## Define the geometry
Define the time domain over which the equation has to be solved, e.g., from $t=0$ to $t=T$.

In [ ]:
T = 
geom = dde.geometry.TimeDomain(0, T)

## Define initial conditions
* Initial condition for Displacement: $x(0)=0$.
* Initial condition for velocity: $v(0)=1$.
* `component=0` refers to the first variable $x(t)$, and `component=1` refers to the second variable $v(t)$.

In [ ]:
ic1 = dde.icbc.IC(geom, lambda t: 0, lambda _, on_initial: on_initial, component=0)
ic2 = dde.icbc.IC(geom, lambda t: 1, lambda _, on_initial: on_initial, component=1)

## Create the Data Object
Combine the geometry, ODE system, and initial conditions into a `Data` object.

In [ ]:
data = dde.data.PDE(geom, ode_system, [ic1, ic2], num_domain=1000, num_boundary=2)

**Explanation**:
* `geom`: The time domain over which the ODE is solved.
* `ode_system`: The function defining the system of ODEs.
* `[ic1, ic2]`: the list of inital conditions for displacement and velocity.
* `num_domain=1000`: The number of collocation points in the domain.
* `num_boundary=2`: The number of boundary points where the initial conditions are enforced.

## Define the neural network model

In [ ]:
net = dde.nn.FNN([1] + [50] * 3 + [2], "tanh", "Glorot uniform")

**Explanation**:
* Network Structure:
    * `[1]`: The input layer with one neuron (time $t$).
    * `[50] * 3`: Three hidden layers with 50 neurons each.
    * `[2]`: The output layer with two neurons (displacement $x(t)$ and velocity $v(t)$).
* `tanh`: The activation function used in the hidden layers, which introduces non-linearity.
* `Glorot unfirom`: The weight initializer that helps in efficient training by appropriately setting inital weights.

## Compile and Train the Model

In [ ]:
model = dde.Model(data, net)
model.compile("adam", lr=0.001, metrics=["l2 relative error"])
losshistory, train_state = model.train(epochs=10000)

**Explanation**:
* `Model(data, net)`: Combines the problem setup and the neural network into a model.
* `compile("adam", lr=0.001, metrics=["l2 relative error"])`: Prepares the model for training using the Adam optimizer with a learning rate of 0.001. The `l2 relative error` metric is used to monitor the model's accuracy.
* `train(epochs=10000)`: Trains the model for 10,000 epochs, iteratively adjusting the network's parameters to minimize the residuals of the ODEs.

## Validate and plot results

In [ ]:
dde.saveplot(losshistory, train_state, issave=True, isplot=True)